# Example Notebook to work with SMA data

In [1]:
import pandas as pd
from sma import SustainabilityMeasurementAgent, Config

config = Config.from_file("examples/minimal.yml")
sma = SustainabilityMeasurementAgent(config)

data = sma.load_measurements()

In [3]:
for df in data:
    display(df.describe())

,feature_node_kubernetes_io_cpu_model_family,feature_node_kubernetes_io_cpu_model_id,feature_node_kubernetes_io_kernel_version_major,feature_node_kubernetes_io_kernel_version_minor,feature_node_kubernetes_io_kernel_version_revision,feature_node_kubernetes_io_system_os_release_VERSION_ID,feature_node_kubernetes_io_system_os_release_VERSION_ID_major,timestamp.1,system_CPU_1,system_CPU_1.1,layer
count,24.0,24.000000,24.0,24.000000,24.0,24.000000,24.000000,2.400000e+01,24.000000,24.000000,0.0
mean,6.0,68.000000,6.0,7.500000,0.0,18.020000,18.000000,1.763396e+09,803.286060,803.286060,NaN
std,0.0,26.559204,0.0,6.639801,0.0,6.149477,6.129047,3.426241e+01,1035.020948,1035.020948,NaN
min,6.0,42.000000,6.0,1.000000,0.0,12.000000,12.000000,1.763396e+09,0.031184,0.031184,NaN
25%,6.0,42.000000,6.0,1.000000,0.0,12.000000,12.000000,1.763396e+09,0.039650,0.039650,NaN
50%,6.0,68.000000,6.0,7.500000,0.0,18.020000,18.000000,1.763396e+09,177.326746,177.326746,NaN
75%,6.0,94.000000,6.0,14.000000,0.0,24.040000,24.000000,1.763396e+09,2232.407880,2232.407880,NaN
max,6.0,94.000000,6.0,14.000000,0.0,24.040000,24.000000,1.763396e+09,2232.655523,2232.655523,NaN


,timestamp.1,node_power_consumption_kepler,node_power_consumption_kepler.1
count,8.000000e+00,8.000000,8.000000
mean,1.763396e+09,17.526230,17.526230
std,3.585686e+01,3.251385,3.251385
min,1.763396e+09,14.159228,14.159228
25%,1.763396e+09,14.604916,14.604916
50%,1.763396e+09,17.583547,17.583547
75%,1.763396e+09,20.589713,20.589713
max,1.763396e+09,20.680918,20.680918


,timestamp.1,wall_power,wall_power.1,layer
count,1.600000e+01,16.000000,16.000000,0.0
mean,1.763396e+09,10.950000,10.950000,NaN
std,3.464102e+01,19.685155,19.685155,NaN
min,1.763396e+09,0.000000,0.000000,NaN
25%,1.763396e+09,0.000000,0.000000,NaN
50%,1.763396e+09,0.000000,0.000000,NaN
75%,1.763396e+09,10.050000,10.050000,NaN
max,1.763396e+09,49.700000,49.700000,NaN


,timestamp.1,system_CPU,system_CPU.1
count,4.000000e+00,4.000000,4.000000
mean,1.763396e+09,0.015257,0.015257
std,3.872983e+01,0.001268,0.001268
min,1.763396e+09,0.014078,0.014078
25%,1.763396e+09,0.014368,0.014368
50%,1.763396e+09,0.015026,0.015026
75%,1.763396e+09,0.015916,0.015916
max,1.763396e+09,0.016898,0.016898


,timestamp.1,pod_power_consumption_kepler_old_watts,pod_power_consumption_kepler_old_watts.1
count,1.680000e+02,168.000000,168.000000
mean,1.763396e+09,0.120643,0.120643
std,3.364129e+01,0.508695,0.508695
min,1.763396e+09,0.000000,0.000000
25%,1.763396e+09,0.000660,0.000660
50%,1.763396e+09,0.006000,0.006000
75%,1.763396e+09,0.014010,0.014010
max,1.763396e+09,3.300120,3.300120


,timestamp.1,node_power_consumption_scaphandre,node_power_consumption_scaphandre.1
count,8.000000e+00,8.000000,8.000000
mean,1.763396e+09,14.970073,14.970073
std,3.585686e+01,0.256947,0.256947
min,1.763396e+09,14.647309,14.647309
25%,1.763396e+09,14.808635,14.808635
50%,1.763396e+09,14.954773,14.954773
75%,1.763396e+09,15.127433,15.127433
max,1.763396e+09,15.344193,15.344193


,timestamp.1,container_info,container_info.1
count,1.600000e+01,16.0,16.0
mean,1.763396e+09,1.0,1.0
std,3.464102e+01,0.0,0.0
min,1.763396e+09,1.0,1.0
25%,1.763396e+09,1.0,1.0
50%,1.763396e+09,1.0,1.0
75%,1.763396e+09,1.0,1.0
max,1.763396e+09,1.0,1.0


,timestamp.1,pod_power_consumption_kepler_watts,pod_power_consumption_kepler_watts.1
count,4.000000e+00,4.000000,4.000000
mean,1.763396e+09,0.255270,0.255270
std,3.872983e+01,0.035207,0.035207
min,1.763396e+09,0.219540,0.219540
25%,1.763396e+09,0.227775,0.227775
50%,1.763396e+09,0.257760,0.257760
75%,1.763396e+09,0.285255,0.285255
max,1.763396e+09,0.286020,0.286020
